In [90]:
import pandas as pd
import numpy as np
from pathlib import Path
import xlsxwriter
import openpyxl
import dateutil
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

def runner(raw_file_name):
    #Dict loading
    aff_number = str(file).split('_')[0]
    dict_frame['AffiliateNumber'].astype('str')
    filtered_dict_frame = dict_frame.loc[dict_frame['AffiliateNumber'] == aff_number]
    dictionary = pd.Series(filtered_dict_frame['MappedTo'].values,index=filtered_dict_frame['RawField']).to_dict()
    print("Loaded dictionary for affiliate {}: Found {} mappings".format(aff_number, len(dictionary)))
    
    #Variables for processing
    workbook = openpyxl.load_workbook(raw_file_name)
    worksheet = workbook.active
    number_of_columns = worksheet.max_column
    number_of_rows = worksheet.max_row - 1
    df = pd.read_excel(raw_file_name, index_col=False, dtype='object')
    count = 0
    no_list = []
    
    # Logic Here
    for heading in df.columns:
        if heading in dictionary:
            df.rename(columns={heading: dictionary[heading]}, inplace=True)
            count = count + 1
        else:
            no_list.append(heading)
        if "Date" in heading:
                df[heading] = pd.to_datetime(df[heading])
                df[heading] = df[heading].dt.strftime('%m/%d/%Y')
            
    #output file variables
    #print(out_df.dtypes)
    filename = (str(raw_file_name).split('.xlsx')[0] + '_DictionaryMapped.xlsx')
    writer = pd.ExcelWriter(filename)
    
    print("Matched and updated {}/{} headers".format(count, number_of_columns))
    print("No matches found for {} headers: {}".format(len(no_list), no_list))
    
    #Write dataframe to excel minus headers
    df.to_excel(writer, engine='xlsxwriter', index=False, startrow=1, header=False)


    #For loop magic to re-add the headers in the right format
    out_workbook  = writer.book
    out_worksheet = writer.sheets['Sheet1']
    column_list = df.columns

    currency_format = out_workbook.add_format({'num_format': '$#,##0.00'})
    date_format = out_workbook.add_format({'num_format': 'm/d/yy'})
    text_format = out_workbook.add_format({'num_format': '@'})

    for idx, val in enumerate(column_list):
        out_worksheet.write(0, idx, val)

    for idx, val in enumerate(column_list):
        if "CopeAmount" in val:
            #print(idx, val)
            out_worksheet.set_column(idx, idx, 10, currency_format)
        if "Date" in val:
            out_worksheet.set_column(idx, idx, 10, date_format)
            #print(idx, val)
        else:
            #print(idx, val)
            out_worksheet.set_column(idx, idx, 10, text_format)


    #Saves the output file to IO
    writer.save()
    

#Open file for dictionary mappings
clean_pathlist = []
pathlist = list((Path('./').glob('**/*.xlsx')))
for path in pathlist:
    if "Dictionary".casefold() in path.stem.casefold():
        if not "~$" in path.stem:
            if not "Mapped" in path.stem:
                clean_pathlist.append(path)

total = len(clean_pathlist)
if total == 1:
    dict_file_name = clean_pathlist[0].resolve()
    dict_frame = pd.read_excel(dict_file_name, index_col=False, dtype='object')
    
    #Open raw file for data access
    clean_pathlist = []
    pathlist = list((Path('./').glob('**/*.xlsx')))
    for path in pathlist:
        if not "Dictionary".casefold() in path.stem.casefold():
            if not "~$" in path.stem:
                clean_pathlist.append(path)

    total = len(clean_pathlist)
    current = 1
    print("------------------------------------")
    for file in clean_pathlist:
        raw_file_name = file.resolve()
        print("Starting file {}/{}: {}".format(current, total, file))
        runner(raw_file_name)
        print("Done running file {}".format(file))
        print("------------------------------------")
        current = current + 1
else:
    print("------------------------------------")
    print("Multiple Dictionary files found, exiting!")
    print("------------------------------------")


------------------------------------
Starting file 1/3: 00300_raw_file.xlsx
Loaded dictionary for affiliate 00300: Found 27 mappings
Matched and updated 4/5 headers
No matches found for 1 headers: ['nothing']
Done running file 00300_raw_file.xlsx
------------------------------------
Starting file 2/3: 06157_34321/06157_raw_file.xlsx
Loaded dictionary for affiliate 06157: Found 13 mappings
Matched and updated 0/1 headers
No matches found for 0 headers: []
Done running file 06157_34321/06157_raw_file.xlsx
------------------------------------
Starting file 3/3: 04671_2323/04671_2312123_raw_file.xlsx
Loaded dictionary for affiliate 04671: Found 17 mappings
Matched and updated 0/1 headers
No matches found for 0 headers: []
Done running file 04671_2323/04671_2312123_raw_file.xlsx
------------------------------------
